# Data Preparation Notebook (Annotating + Filtering)

## Imports and Loading Dataset

In [1]:
import pandas as pd
import os

# Basic
pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")
repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/repository.parquet")
user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/user.parquet")

# Comments and reviews
pr_comments_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_comments.parquet")
pr_reviews_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_reviews.parquet")
pr_review_comments_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_review_comments_v2.parquet")

# Commits
pr_commits_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_commits.parquet")
pr_commit_details_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_commit_details.parquet")

# Related issues
related_issue_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/related_issue.parquet")
issue_df = pd.read_parquet("hf://datasets/hao-li/AIDev/issue.parquet")

# Events
pr_timeline_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_timeline.parquet")

# Task type
pr_task_type_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_task_type.parquet")

# Human-PR
human_pr_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/human_pull_request.parquet")
human_pr_task_type_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/human_pr_task_type.parquet")

print(f"len(pr_df): {len(pr_df)}")
print(f"len(repo_df): {len(repo_df)}")
print(f"len(user_df): {len(user_df)}")

print(f"\nlen(pr_comments_df): {len(pr_comments_df)}")
print(f"len(pr_reviews_df): {len(pr_reviews_df)}")
print(f"len(pr_review_comments_df): {len(pr_review_comments_df)}")

print(f"\nlen(pr_commits_df): {len(pr_commits_df)}")
print(f"len(pr_commit_details_df): {len(pr_commit_details_df)}")


print(f"\nlen(related_issue_df): {len(related_issue_df)}")
print(f"len(issue_df): {len(issue_df)}")

print(f"\nlen(pr_timeline_df): {len(pr_timeline_df)}")

print(f"\nlen(pr_task_type_df): {len(pr_task_type_df)}")

print(f"\nlen(human_pr_df): {len(human_pr_df)}")
print(f"len(human_pr_task_type_df): {len(human_pr_task_type_df)}")

len(pr_df): 33596
len(repo_df): 2807
len(user_df): 1796

len(pr_comments_df): 39122
len(pr_reviews_df): 28875
len(pr_review_comments_df): 26868

len(pr_commits_df): 88576
len(pr_commit_details_df): 711923

len(related_issue_df): 4923
len(issue_df): 4614

len(pr_timeline_df): 325500

len(pr_task_type_df): 33596

len(human_pr_df): 6618
len(human_pr_task_type_df): 6618


## Save Initial Dataset to CSV

In [2]:
def save_dataframes_to_csv(subdirectory):
    # Check if ./data/{subdirectory} directory exists
    data_dir = "./data"
    sub_dir = os.path.join(data_dir, subdirectory)

    if not os.path.exists(sub_dir):
        # Create the directory
        os.makedirs(sub_dir)

        # Save all dataframes to CSV
        pr_df.to_csv(os.path.join(sub_dir, "pull_request.csv"), index=False)
        repo_df.to_csv(os.path.join(sub_dir, "repository.csv"), index=False)
        user_df.to_csv(os.path.join(sub_dir, "user.csv"), index=False)
        pr_comments_df.to_csv(os.path.join(
            sub_dir, "pr_comments.csv"), index=False)
        pr_reviews_df.to_csv(os.path.join(
            sub_dir, "pr_reviews.csv"), index=False)
        pr_review_comments_df.to_csv(os.path.join(
            sub_dir, "pr_review_comments.csv"), index=False)
        pr_commits_df.to_csv(os.path.join(
            sub_dir, "pr_commits.csv"), index=False)
        pr_commit_details_df.to_csv(os.path.join(
            sub_dir, "pr_commit_details.csv"), index=False)
        related_issue_df.to_csv(os.path.join(
            sub_dir, "related_issue.csv"), index=False)
        issue_df.to_csv(os.path.join(sub_dir, "issue.csv"), index=False)
        pr_timeline_df.to_csv(os.path.join(
            sub_dir, "pr_timeline.csv"), index=False)
        pr_task_type_df.to_csv(os.path.join(
            sub_dir, "pr_task_type.csv"), index=False)
        human_pr_df.to_csv(os.path.join(
            sub_dir, "human_pull_request.csv"), index=False)
        human_pr_task_type_df.to_csv(os.path.join(
            sub_dir, "human_pr_task_type.csv"), index=False)

save_dataframes_to_csv("original")

## Annotating Features

New columns added to PR dataframe:

- accepted (bool)
- rejected (bool)
- related_issue (bool)
- turnaround_time (seconds) - number of seconds between creation and decision (merged/closed)
- touches_test_file (bool) - whether "test" appears in any of the filenames modified by the PR
- lines_added (int) - number of lines added throughout all PR commits
- lines_deleted (int) - number of lines deleted throughout all PR commits
- total_churn (int) - total churn across all PR commits (lines added + lines deleted)
- net_churn (int) - net churn across all PR commits (lines added - lines deleted)


TODO:

- num_unique_users - number of users involved in the PR (reviews and comments)
- num_prior_prs - number of PRs submitted to the same repository by the author before the current PR

In [3]:
def annotate_pr_features(pr_dataframe, human=False):
    """
    Annotate PR dataframe with additional features.
    
    Parameters:
    - pr_dataframe: DataFrame containing pull request data
    
    Returns:
    - Annotated PR DataFrame
    """
    # Create a copy to avoid modifying the original
    df = pr_dataframe.copy()
    
    # Drop columns if they already exist
    columns_to_drop = ['accepted', 'rejected', 'pending', 'related_issue', 'turnaround_time', 'num_files_changed',
                       'touches_test_file', 'lines_added', 'lines_deleted', 'net_churn', 'total_churn']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    
    df['accepted'] = df['merged_at'].notnull()
    df['rejected'] = (df['state'] == 'closed') & df['merged_at'].isnull()
    df['turnaround_time'] = (pd.to_datetime(df['closed_at']) - pd.to_datetime(df['created_at'])).dt.total_seconds()


    if not human:
        df['related_issue'] = df['id'].isin(related_issue_df['pr_id'].unique().tolist())
            
        # Number of files changed per PR
        pr_files_changed = pr_commit_details_df.groupby('pr_id')['filename'].nunique().reset_index()
        pr_files_changed.columns = ['id', 'num_files_changed']
        df = df.merge(pr_files_changed, on='id', how='left')
        
        def touches_test_file(filenames):
            for filename in filenames:
                if filename is not None and 'test' in filename.lower():
                    return True
            return False
        
        pr_files = pr_commit_details_df.groupby('pr_id')['filename'].apply(list).reset_index()
        pr_files['touches_test_file'] = pr_files['filename'].apply(touches_test_file)
        df = df.merge(pr_files[['pr_id', 'touches_test_file']], left_on='id', right_on='pr_id', how='left')
        df = df.drop('pr_id', axis=1)
        df['touches_test_file'] = df['touches_test_file'].fillna(False)
        
        # PR Size (number of lines added and deleted across all commits)
        pr_size = pr_commit_details_df.groupby('pr_id').agg({'additions': 'sum', 'deletions': 'sum'}).reset_index()
        pr_size.columns = ['id', 'lines_added', 'lines_deleted']
        df = df.merge(pr_size, on='id', how='left')
        
        # Net Churn & Total Churn
        df['net_churn'] = df['lines_added'] - df['lines_deleted']
        df['total_churn'] = df['lines_added'] + df['lines_deleted']
    
    return df

# Apply annotation to both dataframes
pr_df = annotate_pr_features(pr_df)
human_pr_df = annotate_pr_features(human_pr_df, human=True)

/var/folders/gw/jg8766954y92j4x7kpzdhg680000gp/T/ipykernel_95503/3027076819.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['touches_test_file'] = df['touches_test_file'].fillna(False)


In [4]:
print("Agentic PRs:")
pr_df.head()

Agentic PRs:


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,...,accepted,rejected,turnaround_time,related_issue,num_files_changed,touches_test_file,lines_added,lines_deleted,net_churn,total_churn
0,3264933329,2911,Fix: Wait for all partitions in load_collectio...,## Summary\n\nFixes an issue where `load_colle...,Claude_Code,108661493,weiliu1031,closed,2025-07-26T02:59:01Z,2025-07-29T07:01:20Z,...,False,True,273739.0,False,3.0,True,394.0,2.0,392.0,396.0
1,3265118634,2,ファイルパス参照を相対パスに統一し、doc/からdocs/に統一,## 背景\n\n現在、本プロジェクトにおいて以下のパス構成の不整合が生じています：\n\n...,Claude_Code,61827001,cm-kojimat,closed,2025-07-26T04:56:55Z,2025-07-26T22:12:24Z,...,True,False,62129.0,False,11.0,True,38.0,38.0,0.0,76.0
2,3265640341,30,Add build staleness detection for debug CLI,## Summary\r\n\r\n Implements comprehensive b...,Claude_Code,7475,MSch,closed,2025-07-26T13:31:19Z,2025-07-26T13:37:22Z,...,True,False,363.0,False,5.0,False,298.0,109.0,189.0,407.0
3,3265709660,205,feat: add comprehensive README screenshots wit...,## Type of Change\n\n- [ ] 🐛 `bug` - Bug fix (...,Claude_Code,80381,sugyan,closed,2025-07-26T14:07:22Z,2025-07-26T14:45:30Z,...,True,False,2288.0,False,15.0,False,288.0,12.0,276.0,300.0
4,3265782173,17625,chore: remove HashedPostStateProvider trait,## Summary\r\n\r\n#17545 \r\n\r\nRemove the un...,Claude_Code,47593288,adust09,open,2025-07-26T15:02:48Z,None,...,False,False,NaN,False,21.0,True,53.0,168.0,-115.0,221.0


In [5]:
print("Human PRs:")
human_pr_df.head()

Human PRs:


,id,number,title,user,user_id,state,created_at,closed_at,merged_at,repo_url,html_url,body,agent,accepted,rejected,turnaround_time
0,2336888723,85268,feat(aci): add automations index page,ameliahsu,55610339,closed,2025-02-14T19:04:59Z,2025-02-18T22:42:20Z,2025-02-18T22:42:19Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/85268,https://sentry-j41gpomr5.sentry.dev/automation...,Human,True,False,358641.0
1,2447123365,89131,ref(insights): Make use of `<FeatureBadge>` fo...,ryan953,187460,closed,2025-04-08T23:29:50Z,2025-04-09T15:56:55Z,2025-04-09T15:56:54Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/89131,Using the premade component reduces an import ...,Human,True,False,59225.0
2,2438086945,88748,:bug: fix: update how we fetch workflow_id and...,iamrajjoshi,33237075,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,2025-04-04T15:10:57Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,i realized i made a mistake for how i fetch th...,Human,True,False,63238.0
3,2265431531,83085,fix(org-stats): Require project membership,ArthurKnaus,7033940,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,2025-01-08T08:49:40Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,### Problem\r\n\r\nIf the user is not member o...,Human,True,False,3747.0
4,2332333882,85102,ref(consumers): Rename parallel -> batched-par...,evanpurkhiser,1421724,closed,2025-02-12T21:24:17Z,2025-02-12T22:20:33Z,2025-02-12T22:20:33Z,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/85102,Both crons and uptime consumers have a paralle...,Human,True,False,3376.0


## Filtering

We apply the following filters to the dataset:

- Closed (merged/rejected) PRs only
- At least one interaction that isn't by the author before PR decision

In [6]:
def df_filtering(pr_dataframe):
    """
    Filter PR dataframe.

    Parameters:
    - pr_dataframe: DataFrame containing pull request data

    Returns:
    - Filtered PR DataFrame
    """

    df = pr_dataframe.copy()
    
    # Get PR IDs that exist in this dataframe
    existing_pr_ids = df['id'].unique()

    # PRs with non-author reviews (filter to only PRs in this dataframe)
    relevant_reviews = pr_reviews_df[pr_reviews_df['pr_id'].isin(existing_pr_ids)]
    df_indexed = df.set_index('id')
    non_author_reviews = relevant_reviews[
        relevant_reviews['user'] != df_indexed.loc[relevant_reviews['pr_id'], 'user'].values
    ]['pr_id'].unique()

    # PRs with non-author comments (filter to only PRs in this dataframe)
    relevant_comments = pr_comments_df[pr_comments_df['pr_id'].isin(existing_pr_ids)]
    non_author_comments = relevant_comments[
        relevant_comments['user'] != df_indexed.loc[relevant_comments['pr_id'], 'user'].values
    ]['pr_id'].unique()

    # Union of PRs with non-author reviews and comments
    prs_with_non_author_interaction = set(
        non_author_reviews) | set(non_author_comments)

    # Filter df
    df = df[df['id'].isin(prs_with_non_author_interaction)]
    df = df[df['state'] == 'closed']

    return df

# Filter dataframe
pr_df = df_filtering(pr_df)

In [7]:
print("Num. Agentic PRs:", pr_df.shape)

Num. Agentic PRs: (11962, 24)


In [8]:
pr_reviews_df.columns

Index(['id', 'pr_id', 'user', 'user_type', 'state', 'submitted_at', 'body'], dtype='object')

In [9]:
# Get the list of PR IDs from pr_df
filtered_pr_ids = pr_df['id'].unique()

# Filter all PR-related dataframes
pr_comments_df = pr_comments_df[pr_comments_df['pr_id'].isin(filtered_pr_ids)]
pr_reviews_df = pr_reviews_df[pr_reviews_df['pr_id'].isin(filtered_pr_ids)]
# pr_review_comments_df = pr_review_comments_df[pr_review_comments_df['pull_request_url'].str.extract(r'/(\d+)$')[0].astype(int).isin(filtered_pr_ids)]

filtered_review_ids = pr_reviews_df[pr_reviews_df['pr_id'].isin(filtered_pr_ids)]['id'].unique()
pr_review_comments_df = pr_review_comments_df[pr_review_comments_df['pull_request_review_id'].isin(filtered_review_ids)]


pr_commits_df = pr_commits_df[pr_commits_df['pr_id'].isin(filtered_pr_ids)]
pr_commit_details_df = pr_commit_details_df[pr_commit_details_df['pr_id'].isin(filtered_pr_ids)]
pr_timeline_df = pr_timeline_df[pr_timeline_df['pr_id'].isin(filtered_pr_ids)]
pr_task_type_df = pr_task_type_df[pr_task_type_df['id'].isin(filtered_pr_ids)]
related_issue_df = related_issue_df[related_issue_df['pr_id'].isin(filtered_pr_ids)]

# Get unique repo IDs and user IDs from filtered PRs
filtered_repo_ids = pr_df['repo_id'].unique()
filtered_user_ids = pr_df['user_id'].unique()

# Filter repo and user dataframes
repo_df = repo_df[repo_df['id'].isin(filtered_repo_ids)]
user_df = user_df[user_df['id'].isin(filtered_user_ids)]

# Get unique issue IDs from related_issue_df
filtered_issue_ids = related_issue_df['issue_id'].unique()
issue_df = issue_df[issue_df['id'].isin(filtered_issue_ids)]

print(f"Filtered PR count: {len(pr_df)}")
print(f"Filtered pr_comments_df: {len(pr_comments_df)}")
print(f"Filtered pr_reviews_df: {len(pr_reviews_df)}")
print(f"Filtered pr_review_comments_df: {len(pr_review_comments_df)}")
print(f"Filtered pr_commits_df: {len(pr_commits_df)}")
print(f"Filtered pr_commit_details_df: {len(pr_commit_details_df)}")
print(f"Filtered pr_timeline_df: {len(pr_timeline_df)}")
print(f"Filtered pr_task_type_df: {len(pr_task_type_df)}")
print(f"Filtered related_issue_df: {len(related_issue_df)}")
print(f"Filtered human_pr_df: {len(human_pr_df)}")
print(f"Filtered human_pr_task_type_df: {len(human_pr_task_type_df)}")
print(f"Filtered repo_df: {len(repo_df)}")
print(f"Filtered user_df: {len(user_df)}")
print(f"Filtered issue_df: {len(issue_df)}")

Filtered PR count: 11962
Filtered pr_comments_df: 32706
Filtered pr_reviews_df: 24853
Filtered pr_review_comments_df: 22332
Filtered pr_commits_df: 51430
Filtered pr_commit_details_df: 377263
Filtered pr_timeline_df: 193105
Filtered pr_task_type_df: 11962
Filtered related_issue_df: 3220
Filtered human_pr_df: 6618
Filtered human_pr_task_type_df: 6618
Filtered repo_df: 1623
Filtered user_df: 940
Filtered issue_df: 3084


## Save Filtered Dataset to CSV

In [10]:
save_dataframes_to_csv("filtered")

## Check Filtered Data

In [11]:
test_pr_df = pd.read_csv("./data/filtered/pull_request.csv")

print("Number of rows in test_pr_df:", len(test_pr_df))

test_pr_df.head()

Number of rows in test_pr_df: 11962


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,...,accepted,rejected,turnaround_time,related_issue,num_files_changed,touches_test_file,lines_added,lines_deleted,net_churn,total_churn
0,3264933329,2911,Fix: Wait for all partitions in load_collectio...,## Summary\n\nFixes an issue where `load_colle...,Claude_Code,108661493,weiliu1031,closed,2025-07-26T02:59:01Z,2025-07-29T07:01:20Z,...,False,True,273739.0,False,3.0,True,394.0,2.0,392.0,396.0
1,3265640341,30,Add build staleness detection for debug CLI,## Summary\r\n\r\n Implements comprehensive b...,Claude_Code,7475,MSch,closed,2025-07-26T13:31:19Z,2025-07-26T13:37:22Z,...,True,False,363.0,False,5.0,False,298.0,109.0,189.0,407.0
2,3265709660,205,feat: add comprehensive README screenshots wit...,## Type of Change\n\n- [ ] 🐛 `bug` - Bug fix (...,Claude_Code,80381,sugyan,closed,2025-07-26T14:07:22Z,2025-07-26T14:45:30Z,...,True,False,2288.0,False,15.0,False,288.0,12.0,276.0,300.0
3,3214555104,16658,Add function signature breaking change detector,<details><summary>&#x1F6E0 DevTools &#x1F6E0</...,Claude_Code,17039389,harupy,closed,2025-07-09T05:35:26Z,2025-07-11T05:13:35Z,...,True,False,171489.0,False,3.0,True,620.0,0.0,620.0,620.0
4,3214724259,5489,feat: add comprehensive test coverage for form...,## Summary\n\nThis PR enhances the forms plugi...,Claude_Code,82053242,wtfsayo,closed,2025-07-09T06:43:46Z,2025-07-09T06:44:02Z,...,True,False,16.0,False,6.0,True,1013.0,340.0,673.0,1353.0


In [12]:
test_pr_review_comments_df = pd.read_csv("./data/filtered/pr_review_comments.csv")

print("Number of rows in test_pr_review_comments_df:", len(test_pr_review_comments_df))

test_pr_df.head()

Number of rows in test_pr_review_comments_df: 22332


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,...,accepted,rejected,turnaround_time,related_issue,num_files_changed,touches_test_file,lines_added,lines_deleted,net_churn,total_churn
0,3264933329,2911,Fix: Wait for all partitions in load_collectio...,## Summary\n\nFixes an issue where `load_colle...,Claude_Code,108661493,weiliu1031,closed,2025-07-26T02:59:01Z,2025-07-29T07:01:20Z,...,False,True,273739.0,False,3.0,True,394.0,2.0,392.0,396.0
1,3265640341,30,Add build staleness detection for debug CLI,## Summary\r\n\r\n Implements comprehensive b...,Claude_Code,7475,MSch,closed,2025-07-26T13:31:19Z,2025-07-26T13:37:22Z,...,True,False,363.0,False,5.0,False,298.0,109.0,189.0,407.0
2,3265709660,205,feat: add comprehensive README screenshots wit...,## Type of Change\n\n- [ ] 🐛 `bug` - Bug fix (...,Claude_Code,80381,sugyan,closed,2025-07-26T14:07:22Z,2025-07-26T14:45:30Z,...,True,False,2288.0,False,15.0,False,288.0,12.0,276.0,300.0
3,3214555104,16658,Add function signature breaking change detector,<details><summary>&#x1F6E0 DevTools &#x1F6E0</...,Claude_Code,17039389,harupy,closed,2025-07-09T05:35:26Z,2025-07-11T05:13:35Z,...,True,False,171489.0,False,3.0,True,620.0,0.0,620.0,620.0
4,3214724259,5489,feat: add comprehensive test coverage for form...,## Summary\n\nThis PR enhances the forms plugi...,Claude_Code,82053242,wtfsayo,closed,2025-07-09T06:43:46Z,2025-07-09T06:44:02Z,...,True,False,16.0,False,6.0,True,1013.0,340.0,673.0,1353.0
